In [1]:
%matplotlib inline
import pandas as pd
import os
import json
import pyap
import matplotlib.pyplot as plt
import seaborn as sns

from elasticsearch import (
    Elasticsearch,
    helpers
)
from datetime import datetime
from io import StringIO
from configparser import ConfigParser
from sqlalchemy import create_engine


data_dir = os.path.join(os.pardir,'data')
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

es_creds = get_ini_vals(config_file, 'elasticsearch')
mysql_creds = get_ini_vals(config_file, 'mysql')

### Connect to ES


Newest indices are formatted like

`<issue_name>-YY.MM.DD`

In [3]:
es = Elasticsearch(
    [es_creds['host']],
    http_auth=('',''),
    port = es_creds['port'],
    use_ssl=False
)
print(es.info())

# print all indices
indices=es.indices.get_alias().keys()
print(indices)

{'name': 'pvnM2mO', 'cluster_uuid': 'lf55i5JCSBq8nk_ZrTyTKQ', 'cluster_name': 'elasticsearch', 'tagline': 'You Know, for Search', 'version': {'number': '5.2.1', 'lucene_version': '6.4.1', 'build_date': '2017-02-09T22:05:32.386Z', 'build_snapshot': False, 'build_hash': 'db0d481'}}
dict_keys(['civil_right-17.03.18', 'type', 'n-17.03.18', '17.03.09', 'twitter-ross', 'shakespeare', '17.03.12', 'healthcare-17.03.20', '17.03.14', 'civil_right-17.03.19', '.kibana', '17.03.07', 's-17.03.08', '17.03.06', '17.03.13', 'healthcare-17.03.19', 's-17.03.09', '17.03.15', '17.03.17', '17.03.16', 'civil_right-17.03.20', '17.03.08', '17.03.10', '17.03.11', 'healthcare-17.03.18', 's-17.03.10', 'twitter'])


### Connect To Rszt MySQL

In [4]:
# connect to openatrium database
engine = create_engine(
    """mysql+pymysql://{user}:{password}@{host}:{port}/{db}"""
    .format(user=mysql_creds['user'],
            password=mysql_creds['password'],
            host=mysql_creds['host'],
            port=mysql_creds['port'],
            db=mysql_creds['database']
            )
                      )

conn = engine.connect()

# table fields
rzst_event_header =[
    'elasticsearch_id',
    'title',
    'body_value',
    'location_text',
    'event_datetime_from',
    'event_datetime_to',
    'score',
    'insert_dt',
    'event_type',
    'pri_action_type',
    'sec_action_type'
]

# need to create a flow that parses out data we need first...
insert_vals= dict.fromkeys(rzst_event_header, "")

### Run Set of queries, plot scores of results


We have two main indices in Elasticsearch that are issue-specific

In [49]:
issues = ['civil_right', 'healthcare']
actions = {'charity': ["donat*",
                       "give",
                       "donate",
                       "give support",
                       "financial support"
                      ],
           'protest': ['protest', 
                       'march'
                      ],
           'petition': ['petition', 
                        'sign', 
                        'call'
                       ],
           'gathering': ["meetup",
                         "huddle",
                         "congregate",
                         "join us"
                        ],
           'boycott': ["boycott"],
           'advocate': ["call",
                        "email",
                        "reach out",
                        "senator",
                        "representative",
                        "sign petition",
                        "petition"
                       ],
           'vote': ["vote",
                    "cast your ballot"],
           'townhall': ["town hall",
                        "open office",
                        "town meeting",
                        "townhall",
                        'virtual townhall']
    
          }

In [13]:
# get number of results for one query
format = "%y.%m.%d"
index_ = issues[0] + "-"+datetime.now().date().strftime(format)
print(index_)

civil_right-17.03.19


In [50]:
# form query for one action
query = 'text: "' +  '" OR "'.join(actions['gathering']) + '"'
print(query)

text: "meetup" OR "huddle" OR "congregate" OR "join us"


In [51]:
results = es.search(index=index_,
           q=query,
           size=10000,    # have to manually set this, default is 10
           request_timeout=30)

In [52]:
import json

n_results = len(results['hits']['hits'])
for hit in results['hits']['hits']:
    print(hit['_score'], hit['_source'], hit['_id'])
    print("TEXT: %s \n" % hit['_source']['message'])
    print("USER: %s \n" % hit['_source']['user'])
    print("TIMESTAMP: %s \n" % hit['_source']['@timestamp'])

8.083777 {'urls': ['http://goo.gl/XGy51p', ''], '@timestamp': '2017-03-19T18:58:12.000Z', '@version': '1', 'tags': ['civil right'], 'source': 'http://twitter.com/exa_lemon/status/843537105544515584', 'symbols': [], 'user': 'exa_lemon', 'hashtags': [], 'client': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'message': 'RT @jennymizkz: Local horny women are waiting to meet you online! dont wait, join us 100% Free right now:  https://t.co/0AALAfd2lo… https://t.co/tNbWadVlrp', 'type': 'civil_right', 'user_mentions': [{'id_str': '784695352767250432', 'indices': [3, 14], 'id': 784695352767250432, 'name': 'jenny paek', 'screen_name': 'jennymizkz'}], 'retweeted': False} AVrn87eUMNAj8foZ-HYB
TEXT: RT @jennymizkz: Local horny women are waiting to meet you online! dont wait, join us 100% Free right now:  https://t.co/0AALAfd2lo… https://t.co/tNbWadVlrp 

USER: exa_lemon 

TIMESTAMP: 2017-03-19T18:58:12.000Z 

7.7183886 {'urls': ['http://goo.gl/jkasVh', ''], '@timestamp': '

In [45]:
n_results

0

In [55]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 1.8MB/s ta 0:00:01


In [77]:
import boto3
import csv
import os

from tqdm import tqdm


bucket = "mids-capstone-rzst"
session = boto3.Session(profile_name="berkeley")
s3 = session.client("s3", "us-west-2")

class TwitterQueryAction(object):
    def __init__(self, issue_key):
        self.issue = issue_key
        self.firstline = True
        self.outfile = self.issue + ".csv"
        self.buffer = open(self.outfile, "w")
        self.key_prefix = "/".join(["es_queries", datetime.now().date().isoformat()])
        self.line_count = 0
        
        
    @property
    def query(self):
        return 'message: "' +  '" OR "'.join(actions[self.action]) + '"'
    
    
    @property
    def s3_loc(self):
        return "/".join([self.key_prefix, self.outfile])
    
    
    def _write_tweets(self, results):
        if self.firstline:
            self.firstline = False
            fieldnames = ['issue',
                          'action',
                          'id',
                          'es_score',
                          'tweet_timestamp',
                          'query_timestamp',
                          'tweet_user',
                          'tweet'
                         ]
            self.writer = csv.DictWriter(self.buffer, 
                                        fieldnames=fieldnames
                                       )
            self.writer.writeheader()
        n_results = len(results['hits']['hits'])
        if n_results > 0:
            print("Writing %s results.\n" % n_results)
            for result in tqdm(results['hits']['hits']):
                row = {'issue': self.issue,
                       'action': self.action,
                       'id': result['_id'],
                       'es_score': result['_score'],
                       'tweet_timestamp': result['_source']['@timestamp'],
                       'query_timestamp': self.querytimestamp,
                       'tweet_user': result['_source']['user'],
                       'tweet': result['_source']['message']
                      }
                
                # skip retweets
                if row['tweet'][:2] == "RT":
                    continue
                self.writer.writerow(row)
                self.line_count += 1
        else:
            print("No results to save!")
            return None
        
    
    def run(self, action_key):
        self.action = action_key
        indices=[index for index in es.indices.get_alias().keys() if self.issue in index]
        self.querytimestamp = datetime.now().isoformat()
        for index_ in indices:
            print("Querying index %s" % index_)
            print(self.query)
            results = es.search(index=index_,
                                q=self.query,
                                size=10000,    # have to manually set this, default is 10
                                request_timeout=30)
            self._write_tweets(results)
        
    def stop(self):
        self.buffer.close()
        print("Uploading saved {line_count} results to s3://{bucket}/{s3_loc}".format(
                line_count = self.line_count,
                bucket = bucket,
                s3_loc = self.s3_loc
                )
             )
        s3.upload_file(self.outfile, bucket, "/".join([self.s3_loc
                                                      ])
                      )
        os.remove(self.outfile)
    

In [78]:
for issue in issues:
    tq = TwitterQueryAction(issue)

    for action in actions.keys():
        tq.run(action)
    tq.stop()

print("Done!")

100%|██████████| 17/17 [00:00<00:00, 22635.93it/s]

Querying index civil_right-17.03.18
message: "donat*" OR "give" OR "donate" OR "give support" OR "financial support"
Writing 1 results.

Querying index civil_right-17.03.19
message: "donat*" OR "give" OR "donate" OR "give support" OR "financial support"
Writing 57 results.

Querying index civil_right-17.03.20
message: "donat*" OR "give" OR "donate" OR "give support" OR "financial support"
Writing 17 results.

Querying index civil_right-17.03.18
message: "call" OR "email" OR "reach out" OR "senator" OR "representative" OR "sign petition" OR "petition"
No results to save!
Querying index civil_right-17.03.19
message: "call" OR "email" OR "reach out" OR "senator" OR "representative" OR "sign petition" OR "petition"



100%|██████████| 7/7 [00:00<00:00, 38887.59it/s]

Writing 1585 results.

Querying index civil_right-17.03.20
message: "call" OR "email" OR "reach out" OR "senator" OR "representative" OR "sign petition" OR "petition"
Writing 136 results.

Querying index civil_right-17.03.18
message: "town hall" OR "open office" OR "town meeting" OR "townhall" OR "virtual townhall"
No results to save!
Querying index civil_right-17.03.19
message: "town hall" OR "open office" OR "town meeting" OR "townhall" OR "virtual townhall"
No results to save!
Querying index civil_right-17.03.20
message: "town hall" OR "open office" OR "town meeting" OR "townhall" OR "virtual townhall"
No results to save!
Querying index civil_right-17.03.18
message: "meetup" OR "huddle" OR "congregate" OR "join us"
No results to save!
Querying index civil_right-17.03.19
message: "meetup" OR "huddle" OR "congregate" OR "join us"
Writing 7 results.




100%|██████████| 2/2 [00:00<00:00, 7250.31it/s]


...

Writing 2519 results.

Querying index healthcare-17.03.18
message: "petition" OR "sign" OR "call"
Writing 10 results.

Querying index healthcare-17.03.20
message: "vote" OR "cast your ballot"
Writing 797 results.

Querying index healthcare-17.03.19
message: "vote" OR "cast your ballot"



100%|██████████| 282/282 [00:00<00:00, 50866.29it/s]

Writing 2057 results.

Querying index healthcare-17.03.18
message: "vote" OR "cast your ballot"
Writing 10 results.

Querying index healthcare-17.03.20
message: "protest" OR "march"
Writing 86 results.

Querying index healthcare-17.03.19
message: "protest" OR "march"
Writing 282 results.

Querying index healthcare-17.03.18
message: "protest" OR "march"



100%|██████████| 4/4 [00:00<00:00, 21509.25it/s]


No results to save!
Querying index healthcare-17.03.20
message: "boycott"
Writing 4 results.

Querying index healthcare-17.03.19
message: "boycott"
Writing 4 results.

Querying index healthcare-17.03.18
message: "boycott"
No results to save!
Uploading saved 12908 results to s3://mids-capstone-rzst/es_queries/2017-03-19/healthcare.csv
Done!
